In [189]:

#  QUESTION  1 


from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
df_data_1 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://ComputerSystem.' + name + '/tags.csv')

    
df_data_2 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://ComputerSystem.' + name + '/genome-tags.csv')   
Data3=df_data_1.join(df_data_2,  df_data_1.tag==df_data_2.tag, 'inner').select(df_data_1.userId, df_data_1.movieId, df_data_1.tag, df_data_2.tagId)
Data3.cache()


df_data_3 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://ComputerSystem.' + name + '/genome-scores.csv')
Data4=Data3.join(df_data_3,  (Data3.tagId==df_data_3.tagId) & (Data3.movieId==df_data_3.movieId) , 'inner').select(Data3.userId, Data3.movieId,  Data3.tagId, Data3.tag, df_data_3.relevance )
Data4.cache()
Data5=Data4.select(Data4.userId, pyspark.sql.functions.tanh((Data4.relevance*20)-12))
Data5 =Data5.groupBy("userId").sum().withColumnRenamed("sum(TANH(((relevance * 20) - 12)))", "relevance").cache()

result = Data5.sort('relevance', ascending=[0, 1])
Data6= result.limit(1000).cache()
print(" The top 1000 reliable users:")
Data6.show(1000)


 The top 1000 reliable users:
+------+------------------+
|userId|         relevance|
+------+------------------+
| 88738|5886.2286052266945|
| 58612|1700.3142105607249|
| 52814|1648.8382288692467|
| 11081| 1582.598583854529|
| 10616|1447.9943988488749|
|  9815| 1413.244798182617|
| 68558|1261.1889719076662|
|  1741|1119.2541784879343|
| 25737|1068.4360438182066|
|  4450|1000.3835964912716|
| 27898| 969.4690500707225|
|122523| 961.2716671956218|
|119367| 959.6825546999675|
| 77463| 927.4315434523473|
|124998| 854.3481069572272|
| 77297| 845.1531603150682|
| 28906| 840.9469449647924|
|120937| 797.6761602947419|
|  6431| 665.8821765083233|
|111982| 662.9396577252998|
|132195| 662.4140364061677|
| 10514| 639.5480101354968|
|107711| 607.9753558248358|
|131900| 591.0842834020638|
| 20845| 577.0503608958552|
| 28780| 575.6579425139826|
| 57434| 566.7686207532323|
|130446| 553.6307449052152|
| 72766| 544.8235867476019|
| 49129| 544.6906064447298|
| 68625| 535.7112386908605|
| 42245| 531.45043

In [190]:
#  QUESTION  2

df_data_4 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://ComputerSystem.' + name + '/ratings.csv')
Data7=df_data_4.join(Data6,  df_data_4.userId==Data6.userId, 'inner').select(df_data_4.movieId, (df_data_4.rating*Data6.relevance)).withColumnRenamed("(rating * relevance)", "rank")
Data7.cache()
Data7 =Data7.groupBy("movieId").sum().withColumnRenamed("sum(rank)", "rank")
Data7 = Data7.sort('rank', ascending=[0, 1])
Data7= Data7.limit(20).cache()
#Data7.show()


df_data_5 = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load('swift://ComputerSystem.' + name + '/movies.csv')   
Data8=df_data_5.join(Data7,  df_data_5.movieId==Data7.movieId, 'inner').select(df_data_5.movieId, df_data_5.title, Data7.rank)


print("Best 20 movies from 1000 people's scores")
Data8.show()


Best 20 movies from 1000 people's scores
+-------+--------------------+------------------+
|movieId|               title|              rank|
+-------+--------------------+------------------+
|     32|Twelve Monkeys (a...|343691.13721858023|
|     47|Seven (a.k.a. Se7...|340371.83759733755|
|    260|Star Wars: Episod...|359782.00395323953|
|    296| Pulp Fiction (1994)|415367.08530751546|
|    318|Shawshank Redempt...|409525.17213523365|
|    356| Forrest Gump (1994)|395281.52323548956|
|    593|Silence of the La...| 360313.6505073855|
|   1196|Star Wars: Episod...| 349536.5973937372|
|   1270|Back to the Futur...| 352814.6100077703|
|   2571|  Matrix, The (1999)|471354.55943680345|
|   2762|Sixth Sense, The ...| 345048.8538634933|
|   2858|American Beauty (...|358955.41848492954|
|   2959|   Fight Club (1999)|426152.05775319855|
|   4226|      Memento (2000)|378664.83073142625|
|   4993|Lord of the Rings...|395052.41404762154|
|   5952|Lord of the Rings...| 372978.4285739496|
|   7153|